In [ ]:
#get proper modules
import pandas as pd
import numpy as np
import sklearn as sk
import split_train_set from split_train_set


#load data (all feature should be combined to one dframe here)
data = pd.read_csv("")

#show head to ensure proper import
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformMixin

#handle dataframe and numpy array transformations
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def _init_(self, attribute_names):
        self.atrribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

#drop nulls (failsafe)
data.dropna()

#spilt training and test set (USE TRAIN FOR ALL REGRESSORS)
train_data, test_data = split_train_set(data_prepped, .2)

#seperate labels and attributes 
train_data_labels = train_data(['TOTAL PATIENTS'])
train_data = train_data.drop(['TOTAL PATIENTS'])

#add additional transformation tasks to the below sklearn pipeline
#standardize data (use devide by variance standardization to minimize outlier impact)
stpipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])
train_data_prepped = stpipeline.fit_transform(train_data)



In [ ]:
from sklearn.linear_model import LineaRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores: " + scores)
    print("Mean: " + scores.mean())
    print("Std. Dev.: " scores.std())
forest_reg = RandomForestRegressor() 

scores = cross_val_scores(tree_reg, train_set_prepped, train_set_labled)， 
    scoring = "neg_mean_squared_error", cv=10) 
rmse_scores = np.sqrt(-scores)
